# **MedQA probing (will work for generic MCQ data also)**

Generating probes for Baseline (uses num_dot as label style), Label Style Change, True/False Structured, and a mix of True/False Structured & Label change on uploaded MCQ data with neccessary columns of 'id', 'question', 'options', 'answer' (zero indexed), and inferencing a \<model_name\> on the probed data

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [ ]:
MODEL_NAME = "openai/gpt-oss-20b"
MAX_TOKENS = 10000

In [ ]:
"""
Structural probing on SafetyBench-style MCQ data.

Input CSV columns:
- id
- options                (stringified python list: ["opt0","opt1",...])
- question
- answer                 (zero-indexed int)
- num_of_options         (int)

Probes implemented:
1) Baseline MCQ (model outputs ONLY the option label token: a/b/c..., or 1/2/3..., or i/ii/iii...)
2) Label-change MCQ (same, but with different label styles)
3) TF structured probe (model outputs ONLY: True/False)
4) Mixed: TF + label change (TF prompt with different label styles)

Outputs:
- baseline_mcq.csv
- label_change_mcq.csv
- tf_structured.csv
- mixed_tf_label.csv
- progress.json
"""

import ast
import json
import os
import random
import re
from dataclasses import dataclass
from typing import Callable, Dict, List, Optional, Tuple

import pandas as pd
from groq import Groq


# -----------------------------
# Label styles
# -----------------------------
def make_labeler(style: str) -> Callable[[int], str]:
    """
    style examples:
      - "alpha_paren"  -> a) b) c)
      - "alpha_dot"    -> a. b. c.
      - "num_paren"    -> 1) 2) 3)
      - "num_dot"      -> 1. 2. 3.
      - "roman_paren"  -> i) ii) iii)
      - "roman_dot"    -> i. ii. iii.
    """
    style = style.strip().lower()

    def label_alpha(i: int) -> str:
        return chr(ord("a") + i)

    def label_num(i: int) -> str:
        return str(i + 1)

    def label_roman(i: int) -> str:
        # extend if you need >5
        to_roman_d = {1: "i", 2: "ii", 3: "iii", 4: "iv", 5: "v"}
        return to_roman_d[i + 1]

    if style.startswith("alpha"):
        base = label_alpha
    elif style.startswith("num"):
        base = label_num
    elif style.startswith("roman"):
        base = label_roman
    else:
        raise ValueError(f"Unknown label style: {style}")

    suffix = ")" if style.endswith("paren") else "."
    return lambda i: f"{base(i)}{suffix}"


def make_raw_labeler(style: str) -> Callable[[int], str]:
    """
    Raw token the model must output (no punctuation):
      alpha_* -> a, b, c...
      num_*   -> 1, 2, 3...
      roman_* -> i, ii, iii...
    """
    style = style.strip().lower()

    def raw_alpha(i: int) -> str:
        return chr(ord("a") + i)

    def raw_num(i: int) -> str:
        return str(i + 1)

    def raw_roman(i: int) -> str:
        to_roman_d = {1: "i", 2: "ii", 3: "iii", 4: "iv", 5: "v"}
        return to_roman_d[i + 1]

    if style.startswith("alpha"):
        return raw_alpha
    if style.startswith("num"):
        return raw_num
    if style.startswith("roman"):
        return raw_roman

    raise ValueError(f"Unknown label style: {style}")


def label_family(style: str) -> str:
    style = style.strip().lower()
    if style.startswith("alpha"):
        return "letter"
    if style.startswith("num"):
        return "number"
    if style.startswith("roman"):
        return "roman numeral"
    raise ValueError(f"Unknown label style: {style}")


def join_with_or(items: List[str]) -> str:
    if not items:
        return ""
    if len(items) == 1:
        return items[0]
    if len(items) == 2:
        return f"{items[0]} or {items[1]}"
    return ", ".join(items[:-1]) + f", or {items[-1]}"


# -----------------------------
# Prompt builders
# -----------------------------
def render_options(options: List[str], labeler: Callable[[int], str]) -> str:
    return "\n".join([f"{labeler(i)} {opt}" for i, opt in enumerate(options)])


def build_mcq_prompt(
    question: str,
    options: List[str],
    label_style: str = "num_dot",
) -> str:
    labeler = make_labeler(label_style)  # displayed option labels
    raw = make_raw_labeler(label_style)  # what model outputs
    opt_block = render_options(options, labeler)

    allowed = [raw(i) for i in range(len(options))]
    allowed_str = join_with_or(allowed)
    fam = label_family(label_style)

    return f"""You are answering a multiple choice question.

Rules:
- The question has exactly ONE correct option
- Return ONLY the option {fam} (no punctuation)
- Valid answers are ONLY: {allowed_str}
- Do not add any explanation

<Question>
Question: {question}

Options:
{opt_block}
</Question>
"""


def build_tf_prompt(
    question: str,
    options: List[str],
    claim_index: int,
    label_style: str = "num_dot",
) -> str:
    labeler = make_labeler(label_style)
    opt_block = render_options(options, labeler)

    claim_label = labeler(claim_index)
    claim_text = options[claim_index]

    return f"""You are answering a True/False assertion

Rules:
- Return ONLY True or False
- Do not add any explanation

Question: {question}

Options:
{opt_block}

True or False: the correct answer is {claim_label} {claim_text}
"""


# -----------------------------
# Parsing helpers
# -----------------------------
def parse_options_cell(cell) -> List[str]:
    """
    Accepts:
    - python-list string: '["a","b"]'
    - python repr list: "['a','b']"
    - already-a-list
    """
    if isinstance(cell, list):
        return [str(x) for x in cell]
    if isinstance(cell, str):
        s = cell.strip()
        try:
            val = json.loads(s)
            if isinstance(val, list):
                return [str(x) for x in val]
        except Exception:
            pass
        try:
            val = ast.literal_eval(s)
            if isinstance(val, list):
                return [str(x) for x in val]
        except Exception:
            pass
    raise ValueError(f"Could not parse options cell: {cell!r}")


def parse_bool_tf(text: str) -> Optional[bool]:
    if not text:
        return None
    lines = [ln.strip().lower() for ln in text.splitlines() if ln.strip()]
    if not lines:
        return None
    first = lines[0]
    if first.startswith("true"):
        return True
    if first.startswith("false"):
        return False
    if re.search(r"\btrue\b", first):
        return True
    if re.search(r"\bfalse\b", first):
        return False
    return None


def parse_choice_label(text: str, allowed: List[str]) -> Optional[str]:
    """
    Extract a single allowed label token from model output.
    - Case-insensitive for alpha/roman.
    Returns normalized token (lowercased).
    """
    if not text:
        return None

    allowed_norm = {a.strip().lower() for a in allowed}
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    if not lines:
        return None

    first = lines[0].strip().lower()

    if first in allowed_norm:
        return first

    cleaned = re.sub(r"^[\s\(\[\{]+|[\s\)\]\}\.\):,;]+$", "", first).strip().lower()
    if cleaned in allowed_norm:
        return cleaned

    for tok in sorted(allowed_norm, key=len, reverse=True):
        if re.search(rf"(?<![a-z0-9]){re.escape(tok)}(?![a-z0-9])", first):
            return tok

    blob = "\n".join(lines[:5]).lower()
    for tok in sorted(allowed_norm, key=len, reverse=True):
        if re.search(rf"(?<![a-z0-9]){re.escape(tok)}(?![a-z0-9])", blob):
            return tok

    return None


def label_to_index(label: str, label_style: str, n: int) -> Optional[int]:
    """
    Map raw label token (a/1/i) -> 0-based option index.
    """
    style = label_style.strip().lower()
    lab = (label or "").strip().lower()

    if style.startswith("alpha"):
        if len(lab) == 1 and "a" <= lab <= "z":
            idx = ord(lab) - ord("a")
            return idx if 0 <= idx < n else None
        return None

    if style.startswith("num"):
        if re.fullmatch(r"\d+", lab):
            v = int(lab)
            idx = v - 1
            return idx if 0 <= idx < n else None
        return None

    if style.startswith("roman"):
        roman_map = {"i": 0, "ii": 1, "iii": 2, "iv": 3, "v": 4}
        idx = roman_map.get(lab)
        return idx if idx is not None and 0 <= idx < n else None

    return None


# -----------------------------
# Groq runner
# -----------------------------
@dataclass
class ModelConfig:
    api_key: str
    model: str = MODEL_NAME
    temperature: float = 0.0
    max_tokens: int = MAX_TOKENS


class GroqRunner:
    def __init__(self, cfg: ModelConfig):
        self.client = Groq(api_key=cfg.api_key)
        self.cfg = cfg

    def ask(self, prompt: str) -> str:
        completion = self.client.chat.completions.create(
            model=self.cfg.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.cfg.temperature,
            max_tokens=self.cfg.max_tokens,
        )
        return completion.choices[0].message.content or ""

    def answer_mcq_label(self, prompt: str, label_style: str, n: int) -> Tuple[Optional[str], Optional[int]]:
        raw = make_raw_labeler(label_style)
        allowed = [raw(i) for i in range(n)]
        txt = self.ask(prompt)
        lab = parse_choice_label(txt, allowed)
        if lab is None:
            return None, None
        idx = label_to_index(lab, label_style, n)
        return lab, idx

    def answer_tf(self, prompt: str) -> Optional[bool]:
        txt = self.ask(prompt)
        return parse_bool_tf(txt)


# -----------------------------
# IO helpers
# -----------------------------
def _ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def _load_progress(progress_path: str) -> Dict[str, bool]:
    if os.path.exists(progress_path):
        with open(progress_path, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}


def _save_progress(progress_path: str, progress: Dict[str, bool]) -> None:
    with open(progress_path, "w", encoding="utf-8") as f:
        json.dump(progress, f, indent=2)


def _save_probe_rows(out_path: str, rows: List[Dict], mode: str = "w") -> None:
    df_out = pd.DataFrame(rows)
    if mode == "a" and os.path.exists(out_path):
        df_out.to_csv(out_path, index=False, mode="a", header=False)
    else:
        df_out.to_csv(out_path, index=False)


# -----------------------------
# Main probe runner
# -----------------------------
def run_probes_with_checkpointing(
    input_csv: str,
    out_dir: str,
    api_key: str,
    label_styles: List[str] = None,
    tf_mode: str = "both",  # "true_only" | "false_only" | "both"
    mixed_label_styles: Optional[List[str]] = None,
    save_every: int = 1000,
) -> None:
    """
    Files written:
      - baseline_mcq.csv
      - label_change_mcq.csv
      - tf_structured.csv
      - mixed_tf_label.csv
      - progress.json
    """
    if label_styles is None:
        label_styles = ["num_dot"]
    if mixed_label_styles is None:
        # Assuming default label style for structured tf i.e. num_dot is already
        # performed, other label styles should be used
        mixed_label_styles = ["alpha_dot", "roman_dot"]

    _ensure_dir(out_dir)
    progress_path = os.path.join(out_dir, "progress.json")
    progress = _load_progress(progress_path)

    runner = GroqRunner(ModelConfig(api_key=api_key))
    df = pd.read_csv(input_csv)

    # -------------------------
    # 1) Baseline MCQ
    # -------------------------
    probe_name = "baseline_mcq"
    out_path = os.path.join(out_dir, f"{probe_name}.csv")

    if not progress.get(probe_name, False):
        print(f"\nRunning {probe_name} ...")
        if os.path.exists(out_path):
            os.remove(out_path)

        rows: List[Dict] = []
        baseline_label_style = "num_dot"

        for i, row in enumerate(df.itertuples(index=False), start=1):
            qid = getattr(row, "id")
            question = str(getattr(row, "question"))
            options = parse_options_cell(getattr(row, "options"))
            answer = int(getattr(row, "answer"))
            n = len(options)

            prompt = build_mcq_prompt(question, options, label_style=baseline_label_style)
            pred_label, pred_idx = runner.answer_mcq_label(prompt, label_style=baseline_label_style, n=n)

            rows.append({
                "id": qid,
                "probe": probe_name,
                # "category": getattr(row, "category", None), # or other metadata
                "label_style": baseline_label_style,
                "probed_prompt": prompt,
                "question": question,
                "options": json.dumps(options, ensure_ascii=False),
                "answer_idx": answer,
                "pred_mcq_label": pred_label,
                "pred_mcq_idx": pred_idx,  # mapped for eval
                "is_valid": pred_idx is not None,
                "is_correct": (pred_idx == answer) if pred_idx is not None else None,
            })

            if save_every and (i % save_every == 0):
                _save_probe_rows(out_path, rows, mode="a")
                rows = []
                print(f"  saved {i} rows...")

        if rows:
            _save_probe_rows(out_path, rows, mode="a")

        progress[probe_name] = True
        _save_progress(progress_path, progress)
        print(f"Saved {out_path}")
    else:
        print(f"\nSkipping {probe_name} (already done): {out_path}")

    # -------------------------
    # 2) Label-change MCQ
    # -------------------------
    probe_name = "label_change_mcq"
    out_path = os.path.join(out_dir, f"{probe_name}.csv")

    if not progress.get(probe_name, False):
        print(f"\nRunning {probe_name} ...")
        if os.path.exists(out_path):
            os.remove(out_path)

        rows = []
        count = 0
        for style in label_styles:
            if style == "num_dot":
                continue

            for row in df.itertuples(index=False):
                qid = getattr(row, "id")
                question = str(getattr(row, "question"))
                options = parse_options_cell(getattr(row, "options"))
                answer = int(getattr(row, "answer"))
                n = len(options)

                prompt = build_mcq_prompt(question, options, label_style=style)
                pred_label, pred_idx = runner.answer_mcq_label(prompt, label_style=style, n=n)

                rows.append({
                    "id": qid,
                    "probe": probe_name,
                    # "category": getattr(row, "category", None), # or other metadata
                    "label_style": style,
                    "probed_prompt": prompt,
                    "question": question,
                    "options": json.dumps(options, ensure_ascii=False),
                    "answer_idx": answer,
                    "pred_mcq_label": pred_label,
                    "pred_mcq_idx": pred_idx,
                    "is_valid": pred_idx is not None,
                    "is_correct": (pred_idx == answer) if pred_idx is not None else None,
                })

                count += 1
                if save_every and (count % save_every == 0):
                    _save_probe_rows(out_path, rows, mode="a")
                    rows = []
                    print(f"  saved {count} rows...")

        if rows:
            _save_probe_rows(out_path, rows, mode="a")

        progress[probe_name] = True
        _save_progress(progress_path, progress)
        print(f"Saved {out_path}")
    else:
        print(f"\nSkipping {probe_name} (already done): {out_path}")

    # -------------------------
    # 3) TF structured probe
    # -------------------------
    probe_name = "tf_structured"
    out_path = os.path.join(out_dir, f"{probe_name}.csv")

    if not progress.get(probe_name, False):
        print(f"\nRunning {probe_name} ...")
        if os.path.exists(out_path):
            os.remove(out_path)

        rows = []
        for i, row in enumerate(df.itertuples(index=False), start=1):
            qid = getattr(row, "id")
            question = str(getattr(row, "question"))
            options = parse_options_cell(getattr(row, "options"))
            answer = int(getattr(row, "answer"))
            n = len(options)

            claim_pairs: List[Tuple[int, bool]] = []
            if tf_mode in ("true_only", "both"):
                claim_pairs.append((answer, True))
            if tf_mode in ("false_only", "both"):
                if n > 1:
                    claim_pairs.append(((answer + 1) % n, False))

            for claim_idx, expected_tf in claim_pairs:
                prompt = build_tf_prompt(question, options, claim_idx, label_style="num_dot")
                pred_tf = runner.answer_tf(prompt)

                rows.append({
                    "id": qid,
                    "probe": probe_name,
                    # "category": getattr(row, "category", None), # or other metadata
                    "label_style": "num_dot",
                    "probed_prompt": prompt,
                    "question": question,
                    "options": json.dumps(options, ensure_ascii=False),
                    "answer_idx": answer,
                    "claim_idx": claim_idx,
                    "expected_tf": expected_tf,
                    "pred_tf": pred_tf,
                    "is_valid": pred_tf is not None,
                    "is_correct": (pred_tf == expected_tf) if pred_tf is not None else None,
                })

            if save_every and (i % save_every == 0):
                _save_probe_rows(out_path, rows, mode="a")
                rows = []
                print(f"  saved through question {i}...")

        if rows:
            _save_probe_rows(out_path, rows, mode="a")

        progress[probe_name] = True
        _save_progress(progress_path, progress)
        print(f"Saved {out_path}")
    else:
        print(f"\nSkipping {probe_name} (already done): {out_path}")

    # -------------------------
    # 4) Mixed: TF + label change
    # -------------------------
    probe_name = "mixed_tf_label"
    out_path = os.path.join(out_dir, f"{probe_name}.csv")

    if not progress.get(probe_name, False):
        print(f"\nRunning {probe_name} ...")
        if os.path.exists(out_path):
            os.remove(out_path)

        rows = []
        count = 0
        for style in mixed_label_styles:
            for row in df.itertuples(index=False):
                qid = getattr(row, "id")
                question = str(getattr(row, "question"))
                options = parse_options_cell(getattr(row, "options"))
                answer = int(getattr(row, "answer"))
                n = len(options)

                claim_pairs: List[Tuple[int, bool]] = []
                if tf_mode in ("true_only", "both"):
                    claim_pairs.append((answer, True))
                if tf_mode in ("false_only", "both"):
                    if n > 1:
                        claim_pairs.append(((answer + 1) % n, False))

                for claim_idx, expected_tf in claim_pairs:
                    prompt = build_tf_prompt(question, options, claim_idx, label_style=style)
                    pred_tf = runner.answer_tf(prompt)

                    rows.append({
                        "id": qid,
                        "probe": probe_name,
                        # "category": getattr(row, "category", None), # or other metadata
                        "label_style": style,
                        "probed_prompt": prompt,
                        "question": question,
                        "options": json.dumps(options, ensure_ascii=False),
                        "answer_idx": answer,
                        "claim_idx": claim_idx,
                        "expected_tf": expected_tf,
                        "pred_tf": pred_tf,
                        "is_valid": pred_tf is not None,
                        "is_correct": (pred_tf == expected_tf) if pred_tf is not None else None,
                    })

                    count += 1
                    if save_every and (count % save_every == 0):
                        _save_probe_rows(out_path, rows, mode="a")
                        rows = []
                        print(f"  saved {count} rows...")

        if rows:
            _save_probe_rows(out_path, rows, mode="a")

        progress[probe_name] = True
        _save_progress(progress_path, progress)
        print(f"Saved {out_path}")
    else:
        print(f"\nSkipping {probe_name} (already done): {out_path}")

    print("\nDone.")
    print("Outputs in:", out_dir)


if __name__ == "__main__":
    API_KEY = os.environ["GROQ_API_KEY"]
    INPUT_CSV = "medqa_formatted.csv"
    OUT_DIR = "probe_outputs"

    run_probes_with_checkpointing(
        input_csv=INPUT_CSV,
        out_dir=OUT_DIR,
        api_key=API_KEY,
        label_styles=["num_paren", "alpha_paren", "roman_paren"],
        tf_mode="both",
        mixed_label_styles=["alpha_dot", "roman_dot"],
        save_every=100,
    )


Running baseline_mcq ...
  saved 100 rows...
  saved 200 rows...
  saved 300 rows...
  saved 400 rows...
  saved 500 rows...
  saved 600 rows...
  saved 700 rows...
  saved 800 rows...
  saved 900 rows...
  saved 1000 rows...
  saved 1100 rows...
  saved 1200 rows...
  saved 1300 rows...
  saved 1400 rows...
  saved 1500 rows...
  saved 1600 rows...
  saved 1700 rows...
  saved 1800 rows...
  saved 1900 rows...
  saved 2000 rows...
  saved 2100 rows...
  saved 2200 rows...
  saved 2300 rows...
  saved 2400 rows...
  saved 2500 rows...
  saved 2600 rows...
  saved 2700 rows...
  saved 2800 rows...
  saved 2900 rows...
  saved 3000 rows...
  saved 3100 rows...
  saved 3200 rows...
  saved 3300 rows...
  saved 3400 rows...
  saved 3500 rows...
  saved 3600 rows...
  saved 3700 rows...
  saved 3800 rows...
  saved 3900 rows...
  saved 4000 rows...
  saved 4100 rows...
  saved 4200 rows...
  saved 4300 rows...
  saved 4400 rows...
  saved 4500 rows...
  saved 4600 rows...
  saved 4700 rows